# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Experiment, Workspace

In [ ]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)

## Compute Targets

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core import Environment

In [ ]:
amlCompute_cluster_name = "capstone"

try:
    # Get CPU Cluster object
    aml_compute = ws.compute_targets[amlCompute_cluster_name] 
    print("Using existing Compute Target.")
# Create if CPU Cluster Object does not exist already
except: 
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS2_V2",
                                                                max_nodes = 4,
                                                                idle_seconds_before_scaledown=1800
                                                               )
    
    aml_compute = ComputeTarget.create(ws, amlCompute_cluster_name, provisioning_config)
    aml_compute.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

In [ ]:
env = Environment.get(workspace=ws, name="AzureML-AutoML")

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core import Dataset

In [ ]:
loan_dataset = Dataset.get_by_name(ws, name="loan_dataset")

## Data Preparation

In [ ]:
from azureml.pipeline.core import PipelineData, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

# Model Definition

In [ ]:
from azureml.train.sklearn import SKLearn

In [ ]:
estimator = SKLearn(
    source_directory="./scripts",
    entry_script="train.py",
)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.pipeline.steps import HyperDriveStep
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import BayesianParameterSampling

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
params_space = {
    
}
param_sampling = BayesianParameterSampling(params_space)

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="AUC_Weighted",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=30*len(params_space)
)

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service